# Train a RoBERTa model

In [1]:
import sys

sys.path.append("..")

In [2]:
from transformers import (
    DataCollatorWithPadding,
    RobertaConfig,
    RobertaForSequenceClassification,
    RobertaTokenizerFast,
    TrainingArguments,
)

from adna.pylib import consts
from adna.pylib.bpe_dataset import BPEDataset
from adna.pylib.weighted_trainer import WeightedTrainer

## Data augmentation parameters

In [3]:
REV_COMP_RATE = 0.5

## Training parameters

In [4]:
TRAIN_EPOCHS = 50
LEARNING_RATE = 3e-5
TRAIN_BATCH_SIZE = 192
EVAL_BATCH_SIZE = 192

NUM_HIDDEN_LAYERS = 6
NUM_ATTENTION_HEADS = 6
HIDDEN_SIZE = NUM_ATTENTION_HEADS * 32

MODEL_DIR = 'train'  # Save models to this sub-directory

## Get the tokenizer

In [5]:
tokenizer_path = str(consts.MT_DIR)
tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_path)

## Get the datasets

In [6]:
train_dataset = BPEDataset('train', tokenizer, rev_comp_rate=REV_COMP_RATE)
eval_dataset = BPEDataset('val', tokenizer)

In [7]:
train_dataset[0]

{'input_ids': [0, 590, 3686, 1236, 382, 296, 485, 1225, 1029, 390, 1081, 631, 273, 429, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'label': tensor(0)}

In [8]:
len(train_dataset)

287233

## Adjust weights

In [9]:
WEIGHTS = train_dataset.weights
print(WEIGHTS)
WEIGHTS[0] = 1.0
WEIGHTS[1] = 2.0
WEIGHTS

[0.60151492 2.96269211]


array([1., 2.])

## Build the model

In [10]:
config = RobertaConfig(
    vocab_size=consts.VOCAB_SIZE,
    num_hidden_layers=NUM_HIDDEN_LAYERS,
    type_vocab_size=1,
    num_labels=len(WEIGHTS),
    max_position_embeddings=consts.MAX_LENGTH,
    num_attention_heads=NUM_ATTENTION_HEADS,
    hidden_size=HIDDEN_SIZE,
)

In [11]:
model = RobertaForSequenceClassification(config=config)

## Build the trainer

In [13]:
training_args = TrainingArguments(
    output_dir=consts.MT_DIR / MODEL_DIR,
    overwrite_output_dir=True,
    num_train_epochs=TRAIN_EPOCHS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    seed=23,
)

In [14]:
trainer = WeightedTrainer(
    WEIGHTS,
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

## Train

In [15]:
trainer.train()

/museum/rafe/other/adna/.venv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 287233
  Num Epochs = 50
  Instantaneous batch size per device = 192
  Total train batch size (w. parallel, distributed & accumulation) = 192
  Gradient Accumulation steps = 1
  Total optimization steps = 74850


Epoch,Training Loss,Validation Loss
1,0.357900,0.270802
2,0.268200,0.264468
3,0.252600,0.247132
4,0.237800,0.232183
5,0.226000,0.236701
6,0.214500,0.226463
7,0.204700,0.213935
8,0.195300,0.201208
9,0.190300,0.198009
10,0.183100,0.198405


***** Running Evaluation *****
  Num examples = 95744
  Batch size = 192
Saving model checkpoint to ../data/UF46992/train/checkpoint-1497
Configuration saved in ../data/UF46992/train/checkpoint-1497/config.json
Model weights saved in ../data/UF46992/train/checkpoint-1497/pytorch_model.bin
tokenizer config file saved in ../data/UF46992/train/checkpoint-1497/tokenizer_config.json
Special tokens file saved in ../data/UF46992/train/checkpoint-1497/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 95744
  Batch size = 192
Saving model checkpoint to ../data/UF46992/train/checkpoint-2994
Configuration saved in ../data/UF46992/train/checkpoint-2994/config.json
Model weights saved in ../data/UF46992/train/checkpoint-2994/pytorch_model.bin
tokenizer config file saved in ../data/UF46992/train/checkpoint-2994/tokenizer_config.json
Special tokens file saved in ../data/UF46992/train/checkpoint-2994/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 95744
  

TrainOutput(global_step=74850, training_loss=0.15150078579196152, metrics={'train_runtime': 36094.087, 'train_samples_per_second': 397.895, 'train_steps_per_second': 2.074, 'total_flos': 6.91879669575e+16, 'train_loss': 0.15150078579196152, 'epoch': 50.0})

### It looks like epoch 40 is best

Path = `../data/UF46992/train/checkpoint-59880`